In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=c6fefe66da3a183132938e7f2844ba0466915bc5b53b751c20ad7435a3a31432
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Merge CSV Files") \
    .getOrCreate()

# Read the first CSV file
csv_file1 = '/content/drive/MyDrive/DI_DP/amazon_goodread.csv'
df1 = spark.read.csv(csv_file1, header=True, inferSchema=True)

# Read the second CSV file
csv_file2 = '/content/drive/MyDrive/DI_DP/wonderbk.csv'
df2 = spark.read.csv(csv_file2, header=True, inferSchema=True)

# Perform an inner join on the 'isbn' column
merged_df = df1.join(df2, on='isbn', how='inner').select(df1["isbn"])

# Show the result
merged_df.show()

# Optionally, you can save the merged DataFrame to a new CSV file
output_file = '/content/drive/MyDrive/DI_DP/merged_output.csv'
merged_df.write.csv(output_file, header=True)


+----------+
|      isbn|
+----------+
|0393045218|
|0425176428|
|0440234743|
|0609804618|
|1841721522|
|0061076031|
|0345402871|
|0345417623|
|0375759778|
|0375406328|
|0449005615|
|0061099686|
|0671888587|
|0553582747|
|0425182908|
|0440223571|
|0842342702|
|0440225701|
|0156047624|
|0679865691|
+----------+
only showing top 20 rows



In [5]:
import pandas as pd

part1 = pd.read_csv("/content/drive/MyDrive/DI_DP/merged_output.csv/part-00000-e1c07ef8-1d54-48ab-9748-40bb9bf37895-c000.csv")
part2 = pd.read_csv("/content/drive/MyDrive/DI_DP/merged_output.csv/part-00001-e1c07ef8-1d54-48ab-9748-40bb9bf37895-c000.csv")

In [11]:
len(list(set(part1['isbn'])))

19925

In [14]:
list_isbn_duplicate = list(part1['isbn']) + list(part2['isbn'])

In [15]:
len(list_isbn_duplicate)

28123

In [16]:
list_isbn_duplicate = list(set(list_isbn_duplicate))

In [18]:
len(list_isbn_duplicate)

28107

In [19]:
csv_file1 = '/content/drive/MyDrive/DI_DP/amazon_goodread.csv'
df1 = pd.read_csv(csv_file1)
csv_file2 = '/content/drive/MyDrive/DI_DP/wonderbk.csv'
df2 = pd.read_csv(csv_file2)

<ipython-input-19-a0cc5026fefe>:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(csv_file1)


In [29]:
df1.columns
ISBN = []
BOOK_TITLE = []
AUTHOR = []
PUBLISHER = []
YEAR = []

In [30]:
from tqdm import tqdm

for i in tqdm(range(len(df1))):
  ISBN.append(df1['isbn'][i])
  BOOK_TITLE.append(df1['book_title'][i])
  AUTHOR.append(df1['author'][i])
  PUBLISHER.append(df1['publisher'][i])
  YEAR.append(df1['year'][i])

for i in tqdm(range(len(df2))):
  if df2['isbn'][i] in list_isbn_duplicate:
    continue
  else:
    ISBN.append(df2['isbn'][i])
    BOOK_TITLE.append(df2['book_title'][i])
    AUTHOR.append(df2['author'][i])
    PUBLISHER.append(df2['publisher'][i])
    YEAR.append(df2['year'][i])

data = {
    'isbn': ISBN,
    'book_title': BOOK_TITLE,
    'author': AUTHOR,
    'publisher': PUBLISHER,
    'year': YEAR
}

df = pd.DataFrame(data)

# Write the DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/DI_DP/merge.csv')

100%|██████████| 89678/89678 [04:30<00:00, 331.66it/s]


In [31]:
df.to_csv('/content/drive/MyDrive/DI_DP/merge.csv')

In [32]:
df.head()

,isbn,book_title,author,publisher,year
0,0195153448,Classical Mythology,Mark P. O. Morford,Oxford University Press,2002
1,0002005018,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,2001
2,0060973129,Decision in Normandy,Carlo D'Este,HarperPerennial,1991
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,Farrar Straus Giroux,1999
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,W. W. Norton &amp; Company,1999
